### Import Libraries and Load Environment Variables

In [1]:
import os
import json
import pandas as pd
import traceback
import re
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Load environment variables
load_dotenv()
KEY = os.getenv('GROQ_API_KEY')


### Initialize ChatGroq

In [2]:
# Initialize ChatGroq
chat = ChatGroq(
    api_key=KEY,
    model="llama3-70b-8192",
    temperature=0.3
)


### Define the Response JSON Structure Function

In [6]:
# Define the response JSON structure
def generate_response_json(num_questions):
    return {
        str(i): {
            'mcq': 'multiple choice question',
            'options': {
                'a': 'choice here',
                'b': 'choice here',
                'c': 'choice here',
                'd': 'choice here',
            },
            'correct': 'correct answer',
        } for i in range(1, num_questions + 1)
    }


### Create the Prompt Template

In [7]:
# Create the prompt template
system_message = "You are a helpful assistant."
human_message = """
you are expert in {subject}
Create a quiz with the following specifications:
- Subject: {subject}
- Tone: {tone}
- Number of questions: {num_questions}
- Based on the following text: {text}
- Format the response as JSON with the following structure:
{{
  "quiz": {{
    "title": "Quiz Title",
    "questions": [
      {{
        "question": "Question text",
        "options": ["Option A", "Option B", "Option C", "Option D"],
        "correct": "Correct option"
      }},
      ...
    ]
  }}
}}
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", human_message)])


### Chain the Prompt with ChatGroq

In [8]:
# Chain the prompt with ChatGroq
chain = prompt | chat


### Define Input Variables

In [9]:
# Define input variables
SUBJECT = "Deep Learning"
TONE = "Educational"
NUMBER_OF_QUESTIONS = 7  # You can change this to any desired number


TEXT = """Deep learning is a subset of machine learning that uses neural networks
with multiple layers to model and understand complex patterns in data. By 
leveraging large datasets and powerful computational resources, deep learning 
algorithms can automatically learn to perform tasks such as image and speech 
recognition, natural language processing, and game playing. These neural 
networks, often termed as deep neural networks (DNNs), consist of 
interconnected nodes, or neurons, that process data in a hierarchical manner.
Each layer of the network extracts higher-level features from the raw input,
enabling the model to make more accurate predictions. Key architectures
in deep learning include Convolutional Neural Networks (CNNs) for 
image tasks and Recurrent Neural Networks (RNNs) for sequential data.
"""


### Invoke the Chain

In [10]:
# Invoke the chain
response = chain.invoke({
    'subject': SUBJECT,
    'tone': TONE,
    'num_questions': NUMBER_OF_QUESTIONS,
    'text': TEXT
})


### Print Token Usage and Cost

In [11]:
# Print token usage and cost
response_metadata = response.response_metadata
print(f"Total Tokens: {response_metadata['token_usage']['total_tokens']}")
print(f"Prompt Tokens: {response_metadata['token_usage']['prompt_tokens']}")
print(f"Completion Tokens: {response_metadata['token_usage']['completion_tokens']}")
print(f"Total cost: {response_metadata.get('total_cost', 'N/A')}")


Total Tokens: 779
Prompt Tokens: 296
Completion Tokens: 483
Total cost: N/A


### Process the Quiz Response

In [12]:
# Process the quiz response
quiz = response.content
print("Raw response:")
print(quiz)


Raw response:
Here is the quiz in JSON format:

```
{
  "quiz": {
    "title": "Deep Learning Fundamentals",
    "questions": [
      {
        "question": "What is deep learning a subset of?",
        "options": ["Artificial Intelligence", "Machine Learning", "Neural Networks", "Computer Vision"],
        "correct": "Machine Learning"
      },
      {
        "question": "What is the primary function of each layer in a deep neural network?",
        "options": ["To extract raw input features", "To make predictions", "To extract higher-level features from the raw input", "To process data in a linear manner"],
        "correct": "To extract higher-level features from the raw input"
      },
      {
        "question": "What type of neural network is commonly used for image tasks?",
        "options": ["Recurrent Neural Networks (RNNs)", "Convolutional Neural Networks (CNNs)", "Deep Neural Networks (DNNs)", "Feedforward Neural Networks"],
        "correct": "Convolutional Neural Networks

### Try to Parse the JSON Response

In [13]:
# Try to parse the JSON response
try:
    quiz_data = json.loads(quiz)
except json.JSONDecodeError:
    # If JSON parsing fails, try to extract JSON from the text
    json_match = re.search(r'\{.*\}', quiz, re.DOTALL)
    if json_match:
        try:
            quiz_data = json.loads(json_match.group())
        except json.JSONDecodeError:
            print("Failed to extract valid JSON from the response.")
            exit(1)
    else:
        print("No JSON-like structure found in the response.")
        exit(1)


### Process the Quiz Data

In [14]:
# Process the quiz data
quiz_table_data = []

if 'quiz' in quiz_data and 'questions' in quiz_data['quiz']:
    for question in quiz_data['quiz']['questions']:
        mcq = question.get('question', 'N/A')
        options = "|".join([f"{chr(97 + i)}: {option}" for i, option in enumerate(question.get('options', []))])
        correct = question.get('correct', 'N/A')
        quiz_table_data.append({"MCQ": mcq, 'Choices': options, 'Correct': correct})
else:
    print("Unexpected JSON structure. Please check the model's output format.")
    exit(1)


###  Save the Quiz Data to a CSV File

In [15]:
# Save the quiz data to a CSV file
if quiz_table_data:
    df = pd.DataFrame(quiz_table_data)
    df.to_csv('quiz.csv', index=False)
    print("Quiz data saved to 'quiz.csv'")
else:
    print("No quiz data to save.")


Quiz data saved to 'quiz.csv'


### Print the Processed Quiz Data

In [16]:
# Print the processed quiz data
print("\nProcessed Quiz Data:")
for item in quiz_table_data:
    print(f"Question: {item['MCQ']}")
    print(f"Options: {item['Choices']}")
    print(f"Correct Answer: {item['Correct']}")
    print()



Processed Quiz Data:
Question: What is deep learning a subset of?
Options: a: Artificial Intelligence|b: Machine Learning|c: Neural Networks|d: Computer Vision
Correct Answer: Machine Learning

Question: What is the primary function of each layer in a deep neural network?
Options: a: To extract raw input features|b: To make predictions|c: To extract higher-level features from the raw input|d: To process data in a linear manner
Correct Answer: To extract higher-level features from the raw input

Question: What type of neural network is commonly used for image tasks?
Options: a: Recurrent Neural Networks (RNNs)|b: Convolutional Neural Networks (CNNs)|c: Deep Neural Networks (DNNs)|d: Feedforward Neural Networks
Correct Answer: Convolutional Neural Networks (CNNs)

Question: What is the term used to describe the interconnected nodes in a neural network?
Options: a: Neurons|b: Nodes|c: Connections|d: Layers
Correct Answer: Neurons

Question: What type of data is typically processed by Rec